In [1]:
import pandas as pd
import nltk
import numpy as np
import re
# only for english
# from nltk.tokenize import word_tokenize 
from collections import Counter

fp = "../DATASETS/work_arrangements_development_set.csv"
df = pd.read_csv(fp)

In [2]:

def keyword_classify(text):
    lines = text.split('\n')
    # remote_keywords=['location']
    remote_keywords=['remote','wfh','work-from-home','work-from-home']
    hybrid_keywords=['flexible-w','flexible w','hybrid']
    onsite_keywords=['onsite','on-site']
    
    matched_lines = []
    no_location_info=False
    contain_remote=False
    contain_location = False
    contain_onsite = False
    for line in lines:
      if re.search(re.escape('location'), line, re.IGNORECASE):
        cleaned_line = re.sub(re.escape('location'), '', line, flags=re.IGNORECASE)
        cleaned_line = re.sub(r':', '', cleaned_line)  
        cleaned_line = re.sub(r'\s+', '', cleaned_line)  
        matched_lines.append(cleaned_line)  
        
        contain_location = True
        
        if re.search(re.escape('remote'), cleaned_line, re.IGNORECASE): 
          no_location_info =True
          return 'Remote'
        elif '-' in cleaned_line and len(cleaned_line)<12:
          no_location_info =True
          return 'Remote'
      if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in remote_keywords):
        contain_remote =True
      if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in hybrid_keywords):
        return 'Hybrid'  
      if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in onsite_keywords):
        contain_onsite =True
    
    
    if contain_remote and contain_onsite:    
      return 'Hybrid'   
    if contain_remote:
      return 'Remote'
    if contain_location and contain_onsite:
      return 'OnSite'
    
    return 'OnSite'



In [3]:
df['predicted_WA'] = df.apply(
    lambda row: keyword_classify(row['job_ad']),
    axis=1
)

In [4]:

# Remote
# TP, FP, TN, FN
TP = np.sum((df['predicted_WA'] == 'Remote') & (df['y_true'] == "Remote"))
FP = np.sum((df['predicted_WA'] == 'Remote') & (df['y_true'] != "Remote"))
FN = np.sum((df['predicted_WA'] != 'Remote') & (df['y_true'] == "Remote"))
TN = np.sum((df['predicted_WA'] != 'Remote') & (df['y_true'] != "Remote"))

precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

print("\n🔶 Remote classification results:")
print((FP + FN + TP + TN))
print(f"True Positive (TP): {TP}")
print(f"False Positive (FP): {FP}")
print(f"False Negative (FN): {FN}")
print(f"True Negative (TN): {TN}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

'''
🔶 Remote classification results:
99
True Positive (TP): 32
False Positive (FP): 6
False Negative (FN): 2
True Negative (TN): 59
Precision: 0.8421
Recall: 0.9412
F1 Score: 0.8889
Accuracy: 0.9192
'''


🔶 Remote classification results:
99
True Positive (TP): 32
False Positive (FP): 6
False Negative (FN): 2
True Negative (TN): 59
Precision: 0.8421
Recall: 0.9412
F1 Score: 0.8889
Accuracy: 0.9192


'\n🔶 Remote classification results:\n99\nTrue Positive (TP): 32\nFalse Positive (FP): 6\nFalse Negative (FN): 2\nTrue Negative (TN): 59\nPrecision: 0.8421\nRecall: 0.9412\nF1 Score: 0.8889\nAccuracy: 0.9192\n'

In [5]:
# OnSite
TP = np.sum((df['predicted_WA'] == 'OnSite') & (df['y_true'] == "OnSite"))
FP = np.sum((df['predicted_WA'] == 'OnSite') & (df['y_true'] != "OnSite"))
FN = np.sum((df['predicted_WA'] != 'OnSite') & (df['y_true'] == "OnSite"))
TN = np.sum((df['predicted_WA'] != 'OnSite') & (df['y_true'] != "OnSite"))

precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

print("\n🔶 OnSite classification results:")
print((FP + FN + TP + TN))
print(f"True Positive (TP): {TP}")
print(f"False Positive (FP): {FP}")
print(f"False Negative (FN): {FN}")
print(f"True Negative (TN): {TN}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

'''
🔶 OnSite classification results:
99
True Positive (TP): 36
False Positive (FP): 5
False Negative (FN): 8
True Negative (TN): 50
Precision: 0.8780
Recall: 0.8182
F1 Score: 0.8471
Accuracy: 0.8687
'''



🔶 OnSite classification results:
99
True Positive (TP): 36
False Positive (FP): 5
False Negative (FN): 8
True Negative (TN): 50
Precision: 0.8780
Recall: 0.8182
F1 Score: 0.8471
Accuracy: 0.8687


'\n🔶 OnSite classification results:\n99\nTrue Positive (TP): 36\nFalse Positive (FP): 5\nFalse Negative (FN): 8\nTrue Negative (TN): 50\nPrecision: 0.8780\nRecall: 0.8182\nF1 Score: 0.8471\nAccuracy: 0.8687\n'

In [6]:
# Hybrid
TP = np.sum((df['predicted_WA'] == 'Hybrid') & (df['y_true'] == "Hybrid"))
FP = np.sum((df['predicted_WA'] == 'Hybrid') & (df['y_true'] != "Hybrid"))
FN = np.sum((df['predicted_WA'] != 'Hybrid') & (df['y_true'] == "Hybrid"))
TN = np.sum((df['predicted_WA'] != 'Hybrid') & (df['y_true'] != "Hybrid"))

precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

print("\n🟢 Hybrid classification results:")
print((FP + FN + TP + TN))
print(f"True Positive (TP): {TP}")
print(f"False Positive (FP): {FP}")
print(f"False Negative (FN): {FN}")
print(f"True Negative (TN): {TN}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

'''
🟢 Hybrid classification results:
99
True Positive (TP): 14
False Positive (FP): 6
False Negative (FN): 7
True Negative (TN): 72
Precision: 0.7000
Recall: 0.6667
F1 Score: 0.6829
Accuracy: 0.8687
'''



🟢 Hybrid classification results:
99
True Positive (TP): 14
False Positive (FP): 6
False Negative (FN): 7
True Negative (TN): 72
Precision: 0.7000
Recall: 0.6667
F1 Score: 0.6829
Accuracy: 0.8687


'\n🟢 Hybrid classification results:\n99\nTrue Positive (TP): 14\nFalse Positive (FP): 6\nFalse Negative (FN): 7\nTrue Negative (TN): 72\nPrecision: 0.7000\nRecall: 0.6667\nF1 Score: 0.6829\nAccuracy: 0.8687\n'

In [7]:
# example = df.iloc[2]

# # print(example['job_ad'])

# text=example['job_ad']

# def keyword_classify_example(text):
#     lines = text.split('\n')
#     # remote_keywords=['location']
#     remote_keywords=['remote','wfh','work-from-home','work-from-home']
#     hybrid_keywords=['flexible-w','flexible w','hybrid']
#     onsite_keywords=['onsite','on-site']
    
#     matched_lines = []
#     no_location_info=False
#     contain_remote=False
#     contain_location = False
#     contain_onsite = False
#     for line in lines:
#       if re.search(re.escape('location'), line, re.IGNORECASE):
#         cleaned_line = re.sub(re.escape('location'), '', line, flags=re.IGNORECASE)
#         cleaned_line = re.sub(r':', '', cleaned_line)  
#         cleaned_line = re.sub(r'\s+', '', cleaned_line)  
#         matched_lines.append(cleaned_line)  
        
#         contain_location = True
        
#         if re.search(re.escape('remote'), cleaned_line, re.IGNORECASE): 
#           no_location_info =True
#           return '1Remote'
#         elif '-' in cleaned_line and len(cleaned_line)<12:
#           no_location_info =True
#           return '2Remote'
#       if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in remote_keywords):
#         contain_remote =True
#       if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in hybrid_keywords):
#         return '1Hybrid'  
#       if any(re.search(re.escape(keyword), line, re.IGNORECASE)  for keyword in onsite_keywords):
#         contain_onsite =True
    
#     if contain_remote and contain_onsite:    
#       return '2Hybrid'   
#     if contain_remote:
#       return '3Remote'
#     if contain_location and contain_onsite:
#       return '1OnSite'
    
#     return '2OnSite'


# print(keyword_classify_example(text))

In [8]:

#Accuracy
accuracy = np.mean(df['predicted_WA'] == df['y_true'])
print(f"\nAccuracy: {accuracy:.2%}")

print("\nPrediction vs Ground Truth:\n")
for i, row in df.iterrows():
    predicted = row['predicted_WA']
    expected = row['y_true']
    if predicted != expected:
      print(f"[{i}] ❌ Predicted: {predicted} | Expected: {expected}")
    else:
        print(f"[{i}] ✅ Matched:   {predicted}")


'''
Accuracy: 82.83%
'''


Accuracy: 82.83%

Prediction vs Ground Truth:

[0] ✅ Matched:   Remote
[1] ✅ Matched:   Remote
[2] ❌ Predicted: Remote | Expected: Hybrid
[3] ✅ Matched:   OnSite
[4] ✅ Matched:   OnSite
[5] ✅ Matched:   OnSite
[6] ✅ Matched:   Hybrid
[7] ✅ Matched:   OnSite
[8] ✅ Matched:   Hybrid
[9] ✅ Matched:   Remote
[10] ✅ Matched:   OnSite
[11] ✅ Matched:   OnSite
[12] ✅ Matched:   Hybrid
[13] ✅ Matched:   Remote
[14] ✅ Matched:   Hybrid
[15] ✅ Matched:   OnSite
[16] ✅ Matched:   Remote
[17] ✅ Matched:   OnSite
[18] ✅ Matched:   Remote
[19] ✅ Matched:   OnSite
[20] ✅ Matched:   Remote
[21] ✅ Matched:   Remote
[22] ✅ Matched:   Remote
[23] ✅ Matched:   OnSite
[24] ✅ Matched:   Remote
[25] ✅ Matched:   Remote
[26] ✅ Matched:   OnSite
[27] ✅ Matched:   Remote
[28] ✅ Matched:   OnSite
[29] ✅ Matched:   Remote
[30] ✅ Matched:   Remote
[31] ✅ Matched:   Remote
[32] ❌ Predicted: OnSite | Expected: Hybrid
[33] ❌ Predicted: OnSite | Expected: Hybrid
[34] ✅ Matched:   Remote
[35] ✅ Matched:   OnSite
[36] 

'\nAccuracy: 82.83%\n'